In [1]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 데이터 로드
census = pd.read_csv('./Data/korean-births-stats.csv', index_col=0, parse_dates=True)
census.replace('-', np.nan, inplace=True)
census.index.freq = 'MS'
census.head()

,출생아수,사망자수,혼인건수,이혼건수
1981-01-01,88151.0,NaN,49285.0,1827.0
1981-02-01,93556.0,NaN,34481.0,1687.0
1981-03-01,70421.0,NaN,47843.0,2094.0
1981-04-01,66093.0,NaN,35956.0,2189.0
1981-05-01,68940.0,NaN,35769.0,2059.0


___
## 정상성(Stationarity) 분석
___


* 정의
  * 시계열 데이터가 평균과 분산이 일정한 두 점 사이에서 변하지 않으면, 이를 정상성(Stationary)이 있다고 함.

  * 즉, 어느 시점에서 데이터를 관측하더라도 결과가 동일해야 함.

  * 시계열 모델링, 특히 ARIMA 모델링에서 매우 중요한 가정

* 정상성 검정 방법
  
  * <font color="yellow" style="font-size:1.2em">[증강 디키-풀러 검정(ADF Test)](https://en.wikipedia.org/wiki/Augmented_Dickey-Fuller_test)</font>
  
  * <font color="yellow" style="font-size:1.2em">[KPSS 검정](https://en.wikipedia.org/wiki/KPSS_test)</font>



* 비정상 데이터에서 정상화 방법
  
  * 차분(Differencing)
  
  * 로그 변환(Log Transformation)
  * 제곱근 변환(Square Root Transformation)
  * 추세 제거(Detrending)
  * 계절성 제거(Deseasonalizing)

In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller, kpss

정상성 검정

In [21]:
# helper 함수 정의
def adf_test(series):
    result = adfuller(series)
    print(f'- ADF 통계량: {result[0]}')
    print(f'- p-value: {result[1]} ({'정상성 있음' if result[1] < 0.05 else '정상성 없음'})')
    print(f'- 지연수: {result[2]}')
    print(f'- 관찰수: {result[3]}')
    print('- 임계값:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))
    

def kpss_test(series, **kw):
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    print(f'KPSS 통계량: {statistic}')
    print(f'p-value: {p_value} ({'정상성 있음' if p_value > 0.05 else '정상성 없음'})') 
    # KPSS에서는 p-value가 0.05보다 크면 정상성이 있음 (귀무가설이 데이터에 정상성이 있다고 가정함. ADF에서의 반대)
    print(f'지연수: {n_lags}')
    print('임계값:')
    for key, value in critical_values.items():
        print(f' {key} : {value}')


In [17]:
adf_test(census['출생아수'])

- ADF 통계량: -1.3569591663490597
- p-value: 0.602727803155678 (정상성 없음)
- 지연수: 13
- 관찰수: 505
- 임계값:
	1%: -3.443
	5%: -2.867
	10%: -2.570


In [22]:
kpss_test(census['출생아수'])


KPSS 통계량: 2.974137150518698
p-value: 0.01 (정상성 없음)
지연수: 15
임계값:
 10% : 0.347
 5% : 0.463
 2.5% : 0.574
 1% : 0.739


/var/folders/5y/vwfpdbc96lb5kxd7pmfd9rl00000gn/T/ipykernel_10519/2490681931.py:14: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




___
## 시계열 인과성 분석
___

### [Granger Causality Test(그랜저 인과성 테스트)](https://en.wikipedia.org/wiki/Granger_causality)


#### 기본 개념

* 그랜저 인과관계 테스트는 한 시계열이 다른 시계열을 예측하는 데 유용한지 판단하는 가설 검정.

* 시계열 간의 병렬적인 상관관계를 측정하는 것은 난이도가 쉬움.

* 그랜저 인과성 테스트는, 첫 번째 시계열의 변화가 두 번째 시계열의 행동에 영향을 미쳤다는 인과관계를 추정하게 도와줌. 

* 특정 시계열 데이터의 변화를 보고, 시차를 두고 상관성을 보이는 시계열 데이터의 변화 예측에 매우 유용함.

* 다른 시계열 데이터에 시차적 영향을 미치는 시계열을 그랜저 원인(Granger-causes)이라고 일컬음.

#### 절차

1. **데이터 준비**: 두 개의 시계열 데이터 $ X $와 $ Y $를 준비.

2. **모델 설정**:
   - **모델 1**: $ Y $의 과거 값들로 $ Y $를 예측.
   - **모델 2**: $ Y $의 과거 값들과 $ X $의 과거 값들로 $ Y $를 예측

3. **모델 비교**: 두 모델의 예측 성능을 비교하고, 만약 모델 2가 모델 1보다 $ Y $를 더 잘 예측한다면, $ X $는 $ Y $를 Granger 원인한다고 결론 지음

#### 예시

- $ X $: 어떤 경제 지표 (예: 금리)
- $ Y $: 다른 경제 지표 (예: 주식 시장 지수)

1. **모델 1**: 주식 시장 지수의 과거 값들로 주식 시장 지수를 예측
2. **모델 2**: 주식 시장 지수의 과거 값들과 금리의 과거 값들로 주식 시장 지수를 예측

두 모델의 예측 성능을 비교하여 모델 2가 더 나은 예측을 한다면, 금리는 주식 시장 지수를 Granger 원인한다고 결론 내림

#### 한계
  - 실제 인과 관계를 보장하지 않습니다. 단지 통계적 예측 가능성을 평가합니다.
  - 데이터의 시차 선택에 따라 결과가 달라질 수 있습니다.
  - 외부 요인이나 변수들을 고려하지 않습니다.

In [23]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests

# 예시 데이터 생성
data = {
    '금리': [2.5, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5],
    '주식시장지수': [100, 102, 104, 103, 105, 107, 106, 108, 110, 111]
}
df = pd.DataFrame(data)

# 그랜저 인과성 테스트 수행
max_lag = 2
test_result = grangercausalitytests(df[['주식시장지수', '금리']], max_lag, verbose=True)

# 결과 출력
for lag, result in test_result.items():
    print(f"Lag {lag}:")
    for test, value in result[0].items():
        print(f"  {test}: {value[1]}")


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=7.5860  , p=0.0331  , df_denom=6, df_num=1
ssr based chi2 test:   chi2=11.3789 , p=0.0007  , df=1
likelihood ratio test: chi2=7.3555  , p=0.0067  , df=1
parameter F test:         F=7.5860  , p=0.0331  , df_denom=6, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=10.6633 , p=0.0433  , df_denom=3, df_num=2
ssr based chi2 test:   chi2=56.8710 , p=0.0000  , df=2
likelihood ratio test: chi2=16.7437 , p=0.0002  , df=2
parameter F test:         F=10.6633 , p=0.0433  , df_denom=3, df_num=2
Lag 1:
  ssr_ftest: 0.0331043855296036
  ssr_chi2test: 0.0007428128855561525
  lrtest: 0.006685794353476851
  params_ftest: 0.033104385529602307
Lag 2:
  ssr_ftest: 0.04330713116890235
  ssr_chi2test: 4.4732818682855617e-13
  lrtest: 0.00023129084400794122
  params_ftest: 0.04330713116890107


#### 해석

- **시차 1 4가지 테스트 **:
  - ssr based F test: F=7.5860, p=0.0331
  
  - ssr based chi2 test: chi2=11.3789, p=0.0007
  - likelihood ratio test: chi2=7.3555, p=0.0067
  - parameter F test: F=7.5860, p=0.0331
  - **해석**: 모든 테스트에서 p-값이 유의수준 0.05보다 작으므로, 금리가 주식시장지수에 Granger 원인이 될 수 있음

- **시차 2 4가지 테스트**:
  - ssr based F test: F=10.6633, p=0.0433
  
  - ssr based chi2 test: chi2=56.8710, p=0.0000
  - likelihood ratio test: chi2=16.7437, p=0.0002
  - parameter F test: F=10.6633, p=0.0433
  - **해석**: 모든 테스트에서 p-값이 유의수준 0.05보다 작으므로, 금리가 주식시장지수에 Granger 원인이 될 수 있음

#### # Optional : 각 검정기법 세부 설명(참고하세요)
___
#### 1. SSR 기반 F 검정 (ssr based F test)

**SSR (Sum of Squared Residuals)** 은 모델의 잔차(Residual, 예측값과 실제값의 차이)의 제곱합을 의미함. F 검정은 두 모델의 성능을 비교하는 데 사용됨.

- **기본 개념**: 
  - 귀무가설 $ H_0 $: 추가 변수 $ X $는 예측에 유의미한 기여를 하지 않음.
  - 대립가설 $ H_1 $: 추가 변수 $ X $는 예측에 유의미한 기여를 함.
- **절차**:
  1. 두 개의 회귀 모델을 구축함:
     - **제한된 모델**: $ Y $의 과거 값들로만 $ Y $를 예측함.
     - **전체 모델**: $ Y $의 과거 값들과 $ X $의 과거 값들로 $ Y $를 예측함.
  2. 두 모델의 SSR을 계산함.
  3. F 검정 통계량을 계산함:
     $
     F = \frac{(\text{SSR}_{\text{restricted}} - \text{SSR}_{\text{full}}) / q}{\text{SSR}_{\text{full}} / (n - k)}
     $
     여기서 $ q $는 추가된 파라미터의 수, $ n $은 관측치 수, $ k $는 전체 모델의 파라미터 수임.
  4. 이 F 통계량을 사용하여 p-value를 계산하고, 이를 통해 귀무가설을 기각할 수 있는지 판단함.
___
### 2. SSR 기반 카이제곱 검정 (ssr based chi2 test)

카이제곱 검정은 두 모델의 잔차 제곱합의 차이를 카이제곱 분포를 통해 검정함.

- **기본 개념**:
  - 귀무가설 $ H_0 $: 추가 변수 $ X $는 예측에 유의미한 기여를 하지 않음.
  - 대립가설 $ H_1 $: 추가 변수 $ X $는 예측에 유의미한 기여를 함.
- **절차**:
  1. 두 모델의 SSR을 계산함.
  2. 카이제곱 통계량을 계산함:
     $
     \chi^2 = (\text{SSR}_{\text{restricted}} - \text{SSR}_{\text{full}}) / \sigma^2
     $
     여기서 $ \sigma^2 $는 잔차의 분산임.
  3. 이 카이제곱 통계량을 사용하여 p-value를 계산하고, 이를 통해 귀무가설을 기각할 수 있는지 판단함.
___
### 3. 우도비 검정 (likelihood ratio test)

우도비 검정은 두 모델의 로그우도(log-likelihood)를 비교하는 검정임.

- **기본 개념**:
  - 귀무가설 $ H_0 $: 추가 변수 $ X $는 예측에 유의미한 기여를 하지 않음.
  - 대립가설 $ H_1 $: 추가 변수 $ X $는 예측에 유의미한 기여를 함.
- **절차**:
  1. 제한된 모델과 전체 모델의 로그우도를 계산함.
  2. 우도비 통계량을 계산함:
     $
     LR = -2 (\log L_{\text{restricted}} - \log L_{\text{full}})
     $
  3. 이 우도비 통계량은 카이제곱 분포를 따름. 이를 사용하여 p-value를 계산하고, 귀무가설을 기각할 수 있는지 판단함.
___
### 4. 파라미터 F 검정 (parameter F test)

파라미터 F 검정은 회귀 모델에서 추가된 변수들의 파라미터가 모두 0인지 검정하는 방법임. 이 검정은 SSR 기반 F 검정과 유사함.

- **기본 개념**:
  - 귀무가설 $ H_0 $: 추가된 변수들의 파라미터가 모두 0임.
  - 대립가설 $ H_1 $: 추가된 변수들 중 적어도 하나의 파라미터는 0이 아님.
- **절차**:
  1. 두 개의 회귀 모델을 구축함.
  2. 두 모델의 SSR을 계산함.
  3. F 통계량을 계산하여, p-value를 통해 귀무가설을 기각할 수 있는지 판단함.